In [ ]:
import pandas as pd

In [ ]:
# Constantes
PREVIOUS_RESULTS_USED = 5

In [ ]:
raw_dfs = {
    'races': pd.read_parquet('.data_parquet/races.parquet'),
    'results': pd.read_parquet('.data_parquet/results.parquet'),
    'driver_standings': pd.read_parquet('.data_parquet/driver_standings.parquet'),
    'constructor_standings': pd.read_parquet('.data_parquet/constructor_standings.parquet'),
    'qualifying': pd.read_parquet('.data_parquet/qualifying.parquet')
}
display(raw_dfs['races'])

In [ ]:
races_by_number = raw_dfs['races'][['year', 'round']].sort_values(['year', 'round'])
races_by_number['number'] = range(1, len(races_by_number) + 1)
races_number = races_by_number[['number']]
races_df_with_number = raw_dfs['races'].join(races_by_number['number'], on='raceId')
races_df_with_number = races_df_with_number.set_index('number', append=True)
display(races_df_with_number)

In [ ]:
# Exemplo de query a partir de raceId
display(races_df_with_number.xs(1, level='raceId'))
# Exemplo de query a partir de number
display(races_df_with_number.xs(1, level='number'))

In [ ]:
races_df_with_number.index.get_level_values(0)

In [ ]:
raw_dfs['results']

In [ ]:
raw_dfs["results"].loc[1000]

In [ ]:
import numpy as np

# Parâmetros pensados em um primeiro momento
# - Últimas posições de largada do piloto
# - Últimas posições de chegada do piloto
# - Histórico de classificação do piloto
# - Posição "média" de largada da equipe nas últimas corridas
# - Posição "média" de chegada da equipe nas últimas corridas
# - Histórico de classificação da equipe
result_df = raw_dfs["results"]
driver_standings_df = raw_dfs["driver_standings"]

def query_info_for_race(raceId: int, driverId: int):
    race_not_found = raceId not in races_df_with_number.index.get_level_values(0)
    if race_not_found:
        raise ValueError("Race is not present in the dataset")
    race_row = races_df_with_number.xs(raceId, level='raceId')
    race_number = race_row.index[0]
    last_n_races = range(race_number - PREVIOUS_RESULTS_USED, race_number)
    last_n_racesId = races_df_with_number.loc[(
        slice(None),
        last_n_races
    ), :].index.get_level_values(0)
    last_grids = []
    last_finishes = []
    last_standings = []
    last_grids_team = []
    last_finishes_team = []
    last_standings_team = []
    for prev_raceId in last_n_racesId:
        prev_race_results = result_df.loc[prev_raceId]
        drivers_in_race_driverId = prev_race_results.index
        did_not_result = driverId not in drivers_in_race_driverId
        if did_not_result:
            last_grids.append(None)
            last_finishes.append(None)
        else:
            this_driver_result = prev_race_results.loc[driverId]
            last_grids.append(this_driver_result['grid'])
            prev_race_position = this_driver_result['position']
            last_finishes.append(None if np.isnan(prev_race_position) else int(prev_race_position))
        prev_race_standings = driver_standings_df.loc[prev_raceId]
        did_not_have_standing = driverId not in prev_race_standings.index
        if did_not_have_standing:
            last_standings.append(None)
        else:
            last_standings.append(
                prev_race_standings.loc[driverId]["position"]
            )
    display(last_grids)
    display(last_finishes)
    display(last_standings)


query_info_for_race(10, 1)